<a href="https://colab.research.google.com/github/dhrumil96/BIDS/blob/master/BIDS_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Library

In [1]:
import os
import time
import gc

# data science imports
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# utils import
!pip install -q fuzzywuzzy
from fuzzywuzzy import fuzz

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Importing CSV Files

In [0]:
url = 'https://raw.githubusercontent.com/dhrumil96/BIDS/master/movies.csv'
movies = pd.read_csv(url)

url = 'https://raw.githubusercontent.com/dhrumil96/BIDS/master/ratings.csv'
ratings = pd.read_csv(url)

# First Approach

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
df = pd.merge(movies,ratings,on='movieId')
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [6]:
ratings1 = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings1.head()

,rating
title,
'71 (2014),4.0
'Hellboy': The Seeds of Creation (2004),4.0
'Round Midnight (1986),3.5
'Salem's Lot (2004),5.0
'Til There Was You (1997),4.0


In [7]:
ratings1['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings1.head()

,rating,num of ratings
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


In [8]:
moviemat = df.pivot_table(index='userId',columns='title',values='rating')
moviemat.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
ca_user_ratings = moviemat['Captain America: The Winter Soldier (2014)']

In [10]:
similar_to_ca = moviemat.corrwith(ca_user_ratings)

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


**Corelation of the Movie name Captaian America: The Winter Soldier (2014)**

In [11]:
corr_ca = pd.DataFrame(similar_to_ca,columns=['Correlation'])
corr_ca.dropna(inplace=True)
corr_ca.sort_values('Correlation',ascending=False).head(10)

,Correlation
title,
Runner Runner (2013),1.0
Mr. Bean's Holiday (2007),1.0
She's Out of My League (2010),1.0
French Kiss (1995),1.0
"Devil's Backbone, The (Espinazo del diablo, El) (2001)",1.0
Monkeybone (2001),1.0
Deuce Bigalow: Male Gigolo (1999),1.0
"Cell, The (2000)",1.0
"Fuck You, Goethe (Fack Ju Göhte) (2013)",1.0


In [12]:
corr_ca = corr_ca.join(ratings1['num of ratings'])
corr_ca.head()

,Correlation,num of ratings
title,,
(500) Days of Summer (2009),0.276031,42
*batteries not included (1987),-1.000000,7
10 Cloverfield Lane (2016),-0.334218,14
10 Things I Hate About You (1999),0.739940,54
101 Dalmatians (1996),0.961524,47


In [13]:
corr_ca.sort_values('num of ratings', ascending=False).head()

,Correlation,num of ratings
title,,
Forrest Gump (1994),0.163473,329
"Shawshank Redemption, The (1994)",-0.154635,317
Pulp Fiction (1994),0.398282,307
"Silence of the Lambs, The (1991)",0.301309,279
"Matrix, The (1999)",0.428003,278


In [14]:
corr_ca.sort_values('Correlation', ascending=False).head()

,Correlation,num of ratings
title,,
Runner Runner (2013),1.0,2
Mr. Bean's Holiday (2007),1.0,4
She's Out of My League (2010),1.0,9
French Kiss (1995),1.0,45
"Devil's Backbone, The (Espinazo del diablo, El) (2001)",1.0,10


# Second Approach

In [0]:
class KnnRecommender:
  
    def __init__(self):
              
        self.movie_rating_thres = 0
        self.user_rating_thres = 0
        self.model = NearestNeighbors(5,'brut', metric='cosine', n_jobs=-1)

    def set_filter_params(self, movie_rating_thres, user_rating_thres):
       
        self.movie_rating_thres = movie_rating_thres
        self.user_rating_thres = user_rating_thres
    
  
    def _prep_data(self):
                       
        df_ratings = ratings
        df_movies = movies
        
       
        df_movies_cnt = pd.DataFrame(
            df_ratings.groupby('movieId').size(),
            columns=['count'])
        popular_movies = list(set(df_movies_cnt.query('count >= @self.movie_rating_thres').index))  
        movies_filter = df_ratings.movieId.isin(popular_movies).values

        df_users_cnt = pd.DataFrame(
            df_ratings.groupby('userId').size(),
            columns=['count'])
        active_users = list(set(df_users_cnt.query('count >= @self.user_rating_thres').index))  
        users_filter = df_ratings.userId.isin(active_users).values

        df_ratings_filtered = df_ratings[movies_filter & users_filter]

       
        movie_user_mat = df_ratings_filtered.pivot(
            index='movieId', columns='userId', values='rating').fillna(0)
       
        hashmap = {
            movie: i for i, movie in
            enumerate(list(df_movies.set_index('movieId').loc[movie_user_mat.index].title)) 
        }
       
        movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

      
        del df_movies, df_movies_cnt, df_users_cnt
        del df_ratings, df_ratings_filtered, movie_user_mat
        gc.collect()
        return movie_user_mat_sparse, hashmap

        
    def _inference(self, model, data, hashmap,
                   fav_movie, n_recommendations):
      
       
        model.fit(data)
        
       
        print('You have input movie:', fav_movie)
        
        match_found = []
       
        for title, idx in hashmap.items():
            ratio = fuzz.token_set_ratio(title.lower(), fav_movie.lower())
            if ratio >= 50:
                match_found.append((title, idx, ratio))
                
        if not match_found:
            print('No matchfound')
        else:
            for x in match_found:
              print ('Possible found: {0} '.format([x[0]]))
              idx = match_found[0][1]     
              
        
        
        print('Recommendation system start to make inference')
        print('......\n')
        t0 = time.time()
        distances, indices = model.kneighbors(
            data[idx],
            n_neighbors=n_recommendations+1)
        
        
        raw_recommends = \
            sorted(
                list(
                    zip(
                        indices.squeeze().tolist(),
                        distances.squeeze().tolist()
                    )
                ),
                key=lambda x: x[1]
            )[:0:-1]
        print('It took my system {:.2f}s to make inference \n\
              '.format(time.time() - t0))
       
        return raw_recommends

    def make_recommendations(self, fav_movie, n_recommendations):
        
        
        movie_user_mat_sparse, hashmap = self._prep_data()
        
        raw_recommends = self._inference(
            self.model, movie_user_mat_sparse, hashmap,
            fav_movie, n_recommendations)
      
        reverse_hashmap = {v: k for k, v in hashmap.items()}
        print('Recommendations for {}:'.format(fav_movie))
        for i, (idx, dist) in enumerate(raw_recommends):
            print('{0}: {1}, with distance '
                  'of {2}'.format(i+1, reverse_hashmap[idx], dist))



**Creating Object and Passing Movie Name and Number of how many moives should be recommended**

In the make_recommendations() function, one can pass the any movie name as a first argument and as second argument can pass a number to get that much number of recommendations. 

In [16]:
recommend = KnnRecommender()
recommend.set_filter_params(20, 20)
recommend.make_recommendations('Captain America', 10)

You have input movie: Captain America
Possible found: ['Beavis and Butt-Head Do America (1996)'] 
Possible found: ['American Pie (1999)'] 
Possible found: ['Coming to America (1988)'] 
Possible found: ['Team America: World Police (2004)'] 
Possible found: ['Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan (2006)'] 
Possible found: ['Captain America: The First Avenger (2011)'] 
Possible found: ['Captain Phillips (2013)'] 
Possible found: ['Captain America: The Winter Soldier (2014)'] 
Possible found: ['Captain America: Civil War (2016)'] 
Recommendation system start to make inference
......

It took my system 0.11s to make inference 
              
Recommendations for Captain America:
1: Waterboy, The (1998), with distance of 0.6356085889068339
2: Happy Gilmore (1996), with distance of 0.6327892275203002
3: Grumpier Old Men (1995), with distance of 0.6270324369612559
4: Mighty Ducks, The (1992), with distance of 0.6154003252847093
5: Nutty Professor, T